<a href="https://colab.research.google.com/github/Lukedzc/Bootcamp-ML/blob/main/sekcja_4%3B_21_braki_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import bibliotek

In [1]:

import numpy as np
import pandas as pd
import sklearn

sklearn.__version__

'1.2.2'

Wygenerowanie danych
Brakujące dane:
- usunięcie rekordów z brakami
- uzupełnienie braków stałą wartością (średnia, mediana),
- Wytrenowanie modelu w celu uzupełnienia braków,
- Zastosowanie interpolacji (szeregi czasowe) [metody interpolacji]
- Dodanie nowej kategori np. 'brak' (zmienne kategoryczne)
W celu uzupełnienia braków danych (np. średią, medianą) obliczeń dokonujemy na zbiorze treningowym. Zapobiega to, wyciekowi informacji ze zbioru testowego.

In [2]:

data = {
    'size': ['XL', 'L', 'M', np.nan, 'M', 'M'],
    'color': ['red', 'green', 'blue', 'green', 'red', 'green'],
    'gender': ['female', 'male', np.nan, 'female', 'female', 'male'],
    'price': [199.0, 89.0, np.nan, 129.0, 79.0, 89.0],
    'weight': [500, 450, 300, np.nan, 410, np.nan],
    'bought': ['yes', 'no', 'yes', 'no', 'yes', 'no']
}

df_raw = pd.DataFrame(data=data)
df_raw

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,NaN,NaN,300.0,yes
3,NaN,green,female,129.0,NaN,no
4,M,red,female,79.0,410.0,yes
5,M,green,male,89.0,NaN,no


Utworzenie kopii danych

In [4]:
df = df_raw.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   size    5 non-null      object 
 1   color   6 non-null      object 
 2   gender  5 non-null      object 
 3   price   5 non-null      float64
 4   weight  4 non-null      float64
 5   bought  6 non-null      object 
dtypes: float64(2), object(4)
memory usage: 416.0+ bytes


Sprawdzenie braków

Metoda: zwróci maskę logiczną (gdzie brak danych zwraca True)

df.isnull()


In [5]:

df.isnull()

,size,color,gender,price,weight,bought
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,True,True,False,False
3,True,False,False,False,True,False
4,False,False,False,False,False,False
5,False,False,False,False,True,False


In [6]:

df.isnull().sum() # sumowanie brakujących danych dla poszczególnych kolumn

size      1
color     0
gender    1
price     1
weight    2
bought    0
dtype: int64

In [7]:
df.isnull().sum().sum() #łączna liczba brakujących danych

5

In [8]:

df.isnull().sum() / len(df) #sprawdzamy procentowy brak danych; dzielimy sumę przez długość naszych danych

size      0.166667
color     0.000000
gender    0.166667
price     0.166667
weight    0.333333
bought    0.000000
dtype: float64

Uzupełnienie braków - SimpleImputer

Przykład: uzupełnianie braków w kolumnie 'weight'


In [9]:

df[['weight']]

,weight
0,500.0
1,450.0
2,300.0
3,NaN
4,410.0
5,NaN


In [10]:

from sklearn.impute import SimpleImputer

# strategy: 'mean', 'median', 'most_frequent', 'constant'
imputer = SimpleImputer(missing_values=np.nan, strategy='mean') # rodzaj braków np.nan; strategia - przekazanie braków w formie średniej
imputer.fit(df[['weight']])

SimpleImputer()

In [11]:
imputer.statistics_ #parametr - statistics_ - średnia wartość - 415

array([415.])

In [12]:

imputer.transform(df[['weight']]) # przetransformowanie średniej w miejsce brakujących danych

array([[500.],
       [450.],
       [300.],
       [415.],
       [410.],
       [415.]])

In [13]:

df['weight'] = imputer.transform(df[['weight']]) #przypisanie danych do obiektu data frame
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,NaN,NaN,300.0,yes
3,NaN,green,female,129.0,415.0,no
4,M,red,female,79.0,410.0,yes
5,M,green,male,89.0,415.0,no


In [14]:

imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=99.0) #ustawiamy strategię na constant (stałą wartość, w tym przypadku 99 )
imputer.fit_transform(df[['price']])

array([[199.],
       [ 89.],
       [ 99.],
       [129.],
       [ 79.],
       [ 89.]])

In [15]:

imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='L') #dane kategoryczne; uzupełniamy stałą wartością
imputer.fit_transform(df[['size']])

array([['XL'],
       ['L'],
       ['M'],
       ['L'],
       ['M'],
       ['M']], dtype=object)

In [16]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,NaN,NaN,300.0,yes
3,NaN,green,female,129.0,415.0,no
4,M,red,female,79.0,410.0,yes
5,M,green,male,89.0,415.0,no


In [17]:

imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent') #wstawiamy najczęstszy element który występuje dla danej zmiennej
imputer.fit_transform(df[['size']])

array([['XL'],
       ['L'],
       ['M'],
       ['M'],
       ['M'],
       ['M']], dtype=object)

In [18]:
df = df_raw.copy()
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,NaN,NaN,300.0,yes
3,NaN,green,female,129.0,NaN,no
4,M,red,female,79.0,410.0,yes
5,M,green,male,89.0,NaN,no


In [20]:
df.isnull().sum()

size      1
color     0
gender    1
price     1
weight    2
bought    0
dtype: int64

In [21]:
pd.isnull(df['weight'])

0    False
1    False
2    False
3     True
4    False
5     True
Name: weight, dtype: bool

In [22]:

df[pd.isnull(df['weight'])]

,size,color,gender,price,weight,bought
3,NaN,green,female,129.0,NaN,no
5,M,green,male,89.0,NaN,no


In [23]:

df[~pd.isnull(df['weight'])]

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,NaN,NaN,300.0,yes
4,M,red,female,79.0,410.0,yes


In [24]:
pd.notnull(df['weight'])

0     True
1     True
2     True
3    False
4     True
5    False
Name: weight, dtype: bool

In [25]:

df[pd.notnull(df['weight'])]

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,NaN,NaN,300.0,yes
4,M,red,female,79.0,410.0,yes


In [26]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,NaN,NaN,300.0,yes
3,NaN,green,female,129.0,NaN,no
4,M,red,female,79.0,410.0,yes
5,M,green,male,89.0,NaN,no


In [27]:

df.fillna(value='brak') #wszystkie braki danych zastępujemy słowem 'brak'

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,brak,brak,300.0,yes
3,brak,green,female,129.0,brak,no
4,M,red,female,79.0,410.0,yes
5,M,green,male,89.0,brak,no


In [28]:

df.fillna(value=0.0) #wszystkie braki danych zastępujemy słowem 0.0

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,0.0,0.0,300.0,yes
3,0.0,green,female,129.0,0.0,no
4,M,red,female,79.0,410.0,yes
5,M,green,male,89.0,0.0,no


In [29]:

df['size']

0     XL
1      L
2      M
3    NaN
4      M
5      M
Name: size, dtype: object

In [30]:
df['size'].fillna(value='L', inplace=True) # uzupełniamy 'brak'; poz.3 w kolumnie 'size' - L
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,NaN,NaN,300.0,yes
3,L,green,female,129.0,NaN,no
4,M,red,female,79.0,410.0,yes
5,M,green,male,89.0,NaN,no


In [31]:
df.dropna() # usuwanie wszystkich wierszy z brakiem danych

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
4,M,red,female,79.0,410.0,yes


In [32]:
df = df.dropna()
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
4,M,red,female,79.0,410.0,yes


Braki danych w szeregach czasowych